In [1]:
import requests
from atlassian import Confluence
import time
from getpass import getpass

contentApiUrl = '/rest/api/content'
# Change these based on your instance
confluenceBaseUrl = 'https://confluence.corp.bioagilytix.com/confluence'


In [2]:

username = 'yangyang.liu'
password = getpass("password")
confluence = Confluence(
    url='https://confluence.corp.bioagilytix.com/confluence',
    username=username,
    password=password)


In [3]:
def getAllChildren(parentPageId):
    result = confluence.get_page_by_id(page_id=parentPageId, expand="children")
    ##requestUrl = result['children']['_links']['self']
    requestUrl = "{}{}/search?start=0&limit=9999&cql=parent={}".format(
        confluenceBaseUrl, contentApiUrl, "23005842")
    print(requestUrl)

    requestResponse = requests.get(requestUrl, auth=(username, password))
    print(requestResponse)
    return requestResponse.json()['results']


In [4]:
def isDraft(pageId):

    requestUrl = "https://confluence.corp.bioagilytix.com/confluence/rest/cw/1/content/{}/status".format(
        pageId)

    requestResponse = requests.get(requestUrl, auth=(username, password))
    if (requestResponse.status_code == 200):
        print(requestResponse.json())
        if requestResponse.json()['state']['name'] == "Draft":
            return True
    return False


In [9]:
def getConfiFormEntries(pageId, formName):
    requestUrl = "https://confluence.corp.bioagilytix.com/confluence/ajax/confiforms/rest/filter.action?pageId={}&f={}".format(pageId, formName)
    requestResponse = requests.get(requestUrl, auth=(username, password))
    
    return requestResponse.json()['list']['entry']
    


In [13]:
print(getConfiFormEntries(28607721, 'runinfo'))


[{'recordId': 2, 'createdBy': 'yangyang.liu', 'created': 1641886287889, 'id': 'a178b2cc-71c4-4955-b0e1-e2ded31a69a0', 'fields': {'submitted': True, 'rundesc': 'UMR 106 Cell Preparation for Assay_PTHrP_Method_Worksheet', 'flaskBatchID': 2, 'flaskNumbers': 2, 'runid': 2, 'startdate': 1641859200000, 'class': 'fields', 'worksheetDate': 1642550400000, 'reasonForChange': 'Original Entry'}, 'ownedBy': 'yangyang.liu'}]


In [6]:
def getPageBody(pageId):
    requestUrl = "{}/rest/api/content/{}?expand=body.storage".format(
        confluenceBaseUrl, pageId)
    requestResponse = requests.get(requestUrl, auth=(username, password))
    return requestResponse.json()['body']['storage']['value']

In [7]:
bodyStorage = getPageBody(28607721)


In [33]:
from bs4 import BeautifulSoup



parsed_html = BeautifulSoup(bodyStorage)
form = parsed_html.find('ac:structured-macro', attrs={"ac:name": "confiform"})
print(form.findAll('ac:parameter', attrs={"ac:name": "formName"}))


[<ac:parameter ac:name="formName">runinfo</ac:parameter>, <ac:parameter ac:name="formName">runinfo</ac:parameter>, <ac:parameter ac:name="formName">runinfo</ac:parameter>]


In [8]:
print(bodyStorage)

<p class="auto-cursor-target"><ac:structured-macro ac:name="add-label" ac:schema-version="1" ac:macro-id="9cd4602b-ef19-49b6-af58-84c41fb1d244"><ac:parameter ac:name="labels">worksheet-analyst-peer-review,assay-worksheet</ac:parameter></ac:structured-macro></p><ac:structured-macro ac:name="details" ac:schema-version="1" ac:macro-id="4d3de576-b70a-4a00-8809-33c5842d5996"><ac:rich-text-body><p class="auto-cursor-target"><br /></p><table class="relative-table wrapped"><colgroup><col style="width: 221.859px;" /><col style="width: 1603.14px;" /></colgroup><tbody><tr><th style="text-align: left;"><p><span>Category<br /></span></p></th><td style="text-align: left;"><p><span>Method Worksheet<br /></span></p></td></tr><tr><th style="text-align: left;"><p><span>ZenQMS Title</span></p></th><td style="text-align: left;"><p class="p1">UMR 106 Cell Preparation for Assay_PTHrP_Method_Worksheet1</p></td></tr><tr><th style="text-align: left;"><p><span>Version</span></p></th><td style="text-align: left;